In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, TNK test function

This is the class method for running Xopt.

TNK function
$n=2$ variables:
$x_i \in [0, \pi], i=1,2$

Objectives:
- $f_i(x) = x_i$

Constraints:
- $g_1(x) = -x_1^2 -x_2^2 + 1 + 0.1 \cos\left(16 \arctan \frac{x_1}{x_2}\right) \le 0$
- $g_2(x) = (x_1 - 1/2)^2 + (x_2-1/2)^2 \le 0.5$

In [2]:
# Import the class
from xopt import Xopt

The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
import yaml
# Make a proper input file. 
YAML="""
xopt: {output_path: null, verbose: true}

algorithm:
  name: bayesian_exploration
  options: {n_initial_samples: 5,
            mc_samples: 128,
            use_gpu: False,
            n_steps: 20, 
            verbose: True}
  
simulation: 
  name: test_TNK
  evaluate: xopt.evaluators.test_TNK.evaluate_TNK  
  
vocs:
  name: TNK_test
  description: null
  simulation: test_TNK
  templates: null
  variables:
    x1: [0, 3.14159]
    x2: [0, 3.14159]
  objectives: {y1: None}
  constraints:
    c1: [GREATER_THAN, 0]
    c2: ['LESS_THAN', 0.5]
  linked_variables: {x9: x1}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
# Optional: Connect the function directly
#from xopt.evaluators.test_TNK import evaluate_TNK 
#config['simulation']['evaluate'] = evaluate_TNK 

X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true}
algorithm:
  name: bayesian_exploration
  function: xopt.bayesian_exploration.bayesian_exploration
  options: {n_initial_samples: 5, mc_samples: 128, use_gpu: false, n_steps: 20, verbose: true,
    batch_size: 1, sigma: null, executor: null, custom_model: null, seed: null, return_model: false,
    initial_x: null, eval_args: null}
simulation:
  name: test_TNK
  evaluate: xopt.evaluators.test_TNK.evaluate_TNK
  options: {extra_option: abc}
vocs:
  name: TNK_test
  description: null
  simulation: test_TNK
  templates: null
  variables:
    x1: [0, 3.14159]
    x2: [0, 3.14159]
  objectives: {y1: None}
  constraints:
    c1: [GREATER_THAN, 0]
    c2: [LESS_THAN, 0.5]
  linked_variables: {x9: x1}
  constants: {a: dummy_constant}

In [5]:
# Note that the repr string contains all of the config information
X.random_evaluate()

{'y1': 2.2411965142749635,
 'y2': 3.0797878223987585,
 'c1': 13.58823146547074,
 'c2': 9.687070509920211,
 'some_array': array([1, 2, 3])}

# Run CNSGA

CNSGA is designed to run in parallel with an asynchronous executor as defined in PEP 3148

In [6]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

executor = PoolExecutor()
# This will also work. 
#executor=None

In [7]:
# Change max generations
X.run(executor=executor)

Starting at time 2021-08-03T10:07:18-05:00
tensor([[0., 0.]], dtype=torch.float64)
tensor([[1.5229, 0.0000]], dtype=torch.float64)
tensor([[0.6548, 0.9733]], dtype=torch.float64)
tensor([[0.9487, 0.7883]], dtype=torch.float64)
tensor([[0.3440, 1.0143]], dtype=torch.float64)
tensor([[1.0980, 0.6768]], dtype=torch.float64)
tensor([[0.7863, 1.0526]], dtype=torch.float64)
tensor([[1.1042, 0.2565]], dtype=torch.float64)
tensor([[1.0526, 0.1359]], dtype=torch.float64)
tensor([[1.1645, 0.6377]], dtype=torch.float64)
tensor([[1.0257, 0.0646]], dtype=torch.float64)
tensor([[0.4909, 1.1607]], dtype=torch.float64)
tensor([[0.9340, 1.0173]], dtype=torch.float64)
tensor([[0.6054, 0.8712]], dtype=torch.float64)
tensor([[0.5814, 0.8137]], dtype=torch.float64)
tensor([[0.3039, 1.1452]], dtype=torch.float64)
tensor([[0.0658, 1.0291]], dtype=torch.float64)
tensor([[0.6736, 1.1631]], dtype=torch.float64)
tensor([[1.1836, 0.5916]], dtype=torch.float64)
tensor([[1.0159, 0.9531]], dtype=torch.float64)


In [8]:
# The last population is saved internally:
list(X.population)

TypeError: 'NoneType' object is not iterable

In [ ]:
X.population['outputs']

In [ ]:
# Increase the max_generations, and it will continue where it left off
X.algorithm['options']['max_generations'] = 20
X.run(executor=executor)

# Plot

In [ ]:
# Extract objectives from output
key1, key2 = list(X.vocs['objectives'])

x = [o[key1] for o in X.population['outputs']]
y = [o[key2] for o in X.population['outputs']]

import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(x, y, color='blue')
#ax.set_xlim(X_RANGE)
#ax.set_ylim(Y_RANGE)
ax.set_xlabel(key1)
ax.set_xlabel(key2)
ax.set_aspect('auto')
#ax.set_title(NAME)